In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [17]:
import numpy as np
import pandas as pd

# df = pd.read_csv('./kaggle/input/spaceship-titanic/test.csv')
df = pd.read_csv('./kaggle/input/spaceship-titanic/test_update01.csv')


In [19]:
df.columns
df.isnull().sum()

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_nan', 'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'],
      dtype='object')

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_nan         0
Cabin_num         0
Cabin_port        0
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [11]:
df.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin             0
Cabin_num         0
Cabin_port        0
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [12]:
df = pd.get_dummies(df, columns = ['Cabin'], dummy_na=False)
col1 = df.columns[-9:].to_list()
col2 = df.columns[3:-9].to_list()
col3 = df.columns[:3].to_list()
df_col = col3 + col1 + col2
df = df[df_col]

In [14]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_0.0', 'Cabin_1.0',
       'Cabin_2.0', 'Cabin_3.0', 'Cabin_4.0', 'Cabin_5.0', 'Cabin_6.0',
       'Cabin_7.0', 'Cabin_nan', 'Cabin_num', 'Cabin_port', 'Destination',
       'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'Name'],
      dtype='object')

In [16]:
df.rename(columns={'Cabin_0.0':'Cabin_A','Cabin_1.0':'Cabin_B','Cabin_2.0':'Cabin_C','Cabin_3.0':'Cabin_D',
'Cabin_4.0':'Cabin_E','Cabin_5.0':'Cabin_F','Cabin_6.0':'Cabin_G','Cabin_7.0':'Cabin_T'}, inplace = True)
df.to_csv('./kaggle/input/spaceship-titanic/test_update01.csv',index=False)

In [ ]:
# col1 = df.columns[-9:].to_list()
# col2 = df.columns[3:-9].to_list()
# col3 = df.columns[:3].to_list()
# df_col = col3 + col1 + col2
# df = df[df_col]

In [24]:
# 범주형 데이터 변환
df = pd.read_csv('./kaggle/input/spaceship-titanic/test.csv')

def df_replace(dataframe):
    
    global df_train,df
    df_temp = df
    df = dataframe
    df = df.replace({'55 Cancri e':0,'PSO J318.5-22':1,'TRAPPIST-1e':2})
    df = df.replace({'Earth':0,'Europa':1,'Mars':2})
    df = df.replace({False:0,True:1})

    # Cabin : 번호  
    df['Cabin'] = df['Cabin'].str[0]
    df.insert(4, 'Cabin_num', df_temp['Cabin'].str[2:-2])
    df.insert(5, 'Cabin_port', df_temp['Cabin'].str[-1])
    # df = pd.get_dummies(df, columns = ['Cabin'], dummy_na=True)
    df = df.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7})
    df = df.replace({'P':0,'S':1})

    # col1 = df.columns[-9:].to_list()
    # col2 = df.columns[3:-9].to_list()
    # col3 = df.columns[:3].to_list()
    # df_col = col3 + col1 + col2
    # df = df[df_col]

    ## 범주형 데이터 확인할 수 있게 dict로 만들어 놓은 것
    # Cabin = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
    Cabin_port = {'P':0, 'S':1} #P : 배의 좌현(port side) S: 배의 우현(starboard side) 
    Destination = {'55 Cancri e': 0, 'PSO J318.5-22': 1,'TRAPPIST-1e':2}
    HomePlanet = {'Earth':0,'Europa':1,'Mars':2}
    CryoSleep = {'False':0,'True':1}
    VIP = {'False':0,'True':1}
    Transported = {'False':0,'True':1}


    df = df.astype({'HomePlanet':'float','CryoSleep':'float','Cabin':'float','Cabin_num':'float','Cabin_port':'float',
    'Destination':'float','Age':'float','VIP':'float','RoomService':'float','FoodCourt':'float'
    ,'ShoppingMall':'float','Spa':'float','VRDeck':'float',   
    })
    

df_replace(df)

df_train = df

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib

def fillnullclassifier(column_name):

    global df_update

    df_column_name = df_train[df_train[column_name].isnull()]

    df_train01 = df_train.dropna(subset=[column_name])


    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)
    model = xgb.XGBClassifier(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)
    acc = accuracy_score(y_test,pred)
    print(acc)
    
    df_column_name_null = df_column_name.drop(column_name,axis=1)

    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))

    df_column_name[column_name] = pred

    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnullclassifier('Cabin_port')

0.5358851674641149


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_19000\3591987422.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [33]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
import joblib

def fillnull(column_name):

    global X_train, X_test, y_train, y_test

    # df_FoodCourt =  df_train중에서 FoodCourt null값이 존재하는 열만 추출 
    # -> 훈련된 모델의 값으로 다시 채울 예정
    df_column_name = df_train[df_train[column_name].isnull()]
    #df_train 중에서 FoodCourt 의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
    df_train01 = df_train.dropna(subset=[column_name])
    # df_train
    df_train[df_train[column_name].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)

    # scaler
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # 모델
    model = xgb.XGBRegressor(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    # 모델 저장
    model_path = './' + 'xgb_model/' +column_name +'.model'
    # model.save_model(model_path)
    joblib.dump(model, open(model_path, 'wb'))

    pred = model.predict(X_test)
    score = model.score(X_train,y_train)
    print(score)

    global df_update
    # df_01 은 df_train 중에서 FoodCourt 의 값이 null인 경우의 데이터프레임
    df_column_name_null = df_column_name.drop(column_name,axis=1)
    # 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))
    pred.max()
    pred = np.round(pred)

    # 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
    df_column_name[column_name] = pred
    df_column_name
    
    # null값을 채운 dataframe과 df_train01 합치기 
    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnull('Cabin_num')

0.4795372382725098


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_19000\2933944410.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [34]:
df_update.to_csv('./kaggle/input/spaceship-titanic/test_update01.csv',index=False)

In [ ]:
    # col1 = df.columns[-9:].to_list()
    # col2 = df.columns[3:-9].to_list()
    # col3 = df.columns[:3].to_list()
    # df_col = col3 + col1 + col2
    # df = df[df_col]
    # 'Cabin_A':'float','Cabin_B':'float','Cabin_C':'float','Cabin_D':'float','Cabin_E':'float','Cabin_F':'float',
    # 'Cabin_G':'float','Cabin_T':'float',

In [18]:
df = pd.read_csv('./submission.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
 1   Transported  4277 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 37.7+ KB
